# Three Statement Financial Model

Integrated financial model connecting Income Statement, Balance Sheet, and Cash Flow Statement with DCF valuation.

**Author:** Leonardo Gutierrez Ferrara  
**Date:** 2025

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('dark_background')

## 1. Model Assumptions

In [ ]:
# Key Assumptions
assumptions = {
    'revenue_growth': 0.08,       # 8% annual revenue growth
    'cogs_pct': 0.58,             # COGS as % of revenue
    'sga_pct': 0.20,              # SG&A as % of revenue
    'da_pct': 0.05,               # D&A as % of revenue
    'tax_rate': 0.25,             # Corporate tax rate
    'interest_expense': 20,        # Fixed interest expense ($ millions)
    
    # Working Capital (Days)
    'ar_days': 30,                # Accounts Receivable Days
    'inv_days': 60,               # Inventory Days
    'ap_days': 30,                # Accounts Payable Days
    
    # CapEx & Depreciation
    'capex_pct': 0.05,            # CapEx as % of revenue
    'initial_ppe': 400,           # Starting PP&E
    
    # Capital Structure
    'debt': 200,                  # Total debt ($ millions)
    'common_stock': 100,          # Common stock ($ millions)
    'shares_outstanding': 100,    # Millions of shares
    
    # DCF Assumptions
    'wacc': 0.10,                 # Weighted Average Cost of Capital
    'terminal_growth': 0.025,     # Terminal growth rate
    
    # Dividend Policy
    'dividend_payout': 0.50       # Dividend payout ratio
}

# Historical Data (Year 0)
base_year = {
    'revenue': 1000,
    'cash': 150,
    'retained_earnings': 382
}

# Projection Period
years = ['2024A', '2025E', '2026E', '2027E', '2028E', '2029E']
projection_years = 5

## 2. Income Statement

In [ ]:
def build_income_statement(base_revenue, assumptions, years):
    """Build projected income statement"""
    
    n_years = len(years)
    
    # Initialize arrays
    revenue = np.zeros(n_years)
    cogs = np.zeros(n_years)
    gross_profit = np.zeros(n_years)
    sga = np.zeros(n_years)
    da = np.zeros(n_years)
    ebit = np.zeros(n_years)
    interest = np.zeros(n_years)
    ebt = np.zeros(n_years)
    taxes = np.zeros(n_years)
    net_income = np.zeros(n_years)
    
    # Base year
    revenue[0] = base_revenue
    
    # Project revenues
    for i in range(1, n_years):
        revenue[i] = revenue[i-1] * (1 + assumptions['revenue_growth'])
    
    # Calculate P&L items
    for i in range(n_years):
        cogs[i] = -revenue[i] * assumptions['cogs_pct']
        gross_profit[i] = revenue[i] + cogs[i]
        sga[i] = -revenue[i] * assumptions['sga_pct']
        da[i] = -revenue[i] * assumptions['da_pct']
        ebit[i] = gross_profit[i] + sga[i] + da[i]
        interest[i] = -assumptions['interest_expense']
        ebt[i] = ebit[i] + interest[i]
        taxes[i] = -ebt[i] * assumptions['tax_rate']
        net_income[i] = ebt[i] + taxes[i]
    
    # Create DataFrame
    is_df = pd.DataFrame({
        'Revenue': revenue,
        'COGS': cogs,
        'Gross Profit': gross_profit,
        'SG&A': sga,
        'D&A': da,
        'EBIT': ebit,
        'Interest Expense': interest,
        'EBT': ebt,
        'Taxes': taxes,
        'Net Income': net_income
    }, index=years).T
    
    return is_df, revenue, da, net_income

income_statement, revenue, da, net_income = build_income_statement(
    base_year['revenue'], assumptions, years
)

print("INCOME STATEMENT (USD millions)")
print("=" * 60)
income_statement.round(0)

## 3. Balance Sheet

In [ ]:
def build_balance_sheet(revenue, da, net_income, assumptions, base_year, years):
    """Build projected balance sheet"""
    
    n_years = len(years)
    
    # Initialize arrays
    cash = np.zeros(n_years)
    ar = np.zeros(n_years)
    inventory = np.zeros(n_years)
    ppe = np.zeros(n_years)
    total_assets = np.zeros(n_years)
    
    ap = np.zeros(n_years)
    debt = np.zeros(n_years)
    retained_earnings = np.zeros(n_years)
    common_stock = np.zeros(n_years)
    total_le = np.zeros(n_years)
    
    # Calculate working capital items based on days
    ar = revenue * (assumptions['ar_days'] / 365)
    cogs_positive = revenue * assumptions['cogs_pct']
    inventory = cogs_positive * (assumptions['inv_days'] / 365)
    ap = cogs_positive * (assumptions['ap_days'] / 365)
    
    # CapEx and PP&E
    capex = revenue * assumptions['capex_pct']
    ppe[0] = assumptions['initial_ppe']
    for i in range(1, n_years):
        ppe[i] = ppe[i-1] - da[i] + capex[i]
    
    # Debt and Equity
    debt[:] = assumptions['debt']
    common_stock[:] = assumptions['common_stock']
    
    # Retained Earnings
    retained_earnings[0] = base_year['retained_earnings']
    dividends = net_income * assumptions['dividend_payout']
    for i in range(1, n_years):
        retained_earnings[i] = retained_earnings[i-1] + net_income[i] - dividends[i]
    
    # Cash (plug to balance)
    cash[0] = base_year['cash']
    for i in range(1, n_years):
        total_le_temp = ap[i] + debt[i] + retained_earnings[i] + common_stock[i]
        total_assets_excash = ar[i] + inventory[i] + ppe[i]
        cash[i] = total_le_temp - total_assets_excash
    
    # Totals
    total_assets = cash + ar + inventory + ppe
    total_le = ap + debt + retained_earnings + common_stock
    
    # Check balance
    balance_check = total_assets - total_le
    
    # Create DataFrame
    bs_df = pd.DataFrame({
        'Cash': cash,
        'Accounts Receivable': ar,
        'Inventory': inventory,
        'PP&E (Net)': ppe,
        'Total Assets': total_assets,
        '---': ['---'] * n_years,
        'Accounts Payable': ap,
        'Debt': debt,
        'Retained Earnings': retained_earnings,
        'Common Stock': common_stock,
        'Total L&E': total_le,
        'Balance Check': balance_check
    }, index=years).T
    
    return bs_df, cash, ar, inventory, ap, capex, dividends

balance_sheet, cash, ar, inventory, ap, capex, dividends = build_balance_sheet(
    revenue, da, net_income, assumptions, base_year, years
)

print("BALANCE SHEET (USD millions)")
print("=" * 60)
balance_sheet.round(0)

## 4. Cash Flow Statement

In [ ]:
def build_cash_flow(net_income, da, ar, inventory, ap, capex, dividends, years):
    """Build cash flow statement"""
    
    n_years = len(years)
    
    # Working capital changes
    delta_ar = np.zeros(n_years)
    delta_inv = np.zeros(n_years)
    delta_ap = np.zeros(n_years)
    
    for i in range(1, n_years):
        delta_ar[i] = -(ar[i] - ar[i-1])
        delta_inv[i] = -(inventory[i] - inventory[i-1])
        delta_ap[i] = ap[i] - ap[i-1]
    
    delta_wc = delta_ar + delta_inv + delta_ap
    
    # Cash Flow from Operations
    cfo = net_income + np.abs(da) + delta_wc
    
    # Cash Flow from Investing
    cfi = -capex
    
    # Cash Flow from Financing
    cff = -dividends
    
    # Net change in cash
    net_change = cfo + cfi + cff
    
    # Create DataFrame (exclude base year for display)
    cf_df = pd.DataFrame({
        'Net Income': net_income[1:],
        'D&A (add back)': np.abs(da[1:]),
        'Δ Working Capital': delta_wc[1:],
        'Cash from Operations': cfo[1:],
        '---': ['---'] * (n_years-1),
        'CapEx': cfi[1:],
        'Cash from Investing': cfi[1:],
        '----': ['---'] * (n_years-1),
        'Dividends': -dividends[1:],
        'Cash from Financing': cff[1:],
        '-----': ['---'] * (n_years-1),
        'Net Change in Cash': net_change[1:]
    }, index=years[1:]).T
    
    return cf_df, cfo

cash_flow, cfo = build_cash_flow(
    net_income, da, ar, inventory, ap, capex, dividends, years
)

print("CASH FLOW STATEMENT (USD millions)")
print("=" * 60)
cash_flow.round(0)

## 5. DCF Valuation

In [ ]:
def dcf_valuation(revenue, assumptions, years):
    """Calculate enterprise value using DCF"""
    
    wacc = assumptions['wacc']
    g = assumptions['terminal_growth']
    tax_rate = assumptions['tax_rate']
    
    # Calculate Unlevered Free Cash Flow (UFCF)
    ebit = revenue * (1 - assumptions['cogs_pct'] - assumptions['sga_pct'] - assumptions['da_pct'])
    nopat = ebit * (1 - tax_rate)
    da = revenue * assumptions['da_pct']
    capex = revenue * assumptions['capex_pct']
    
    # NWC changes
    ar = revenue * (assumptions['ar_days'] / 365)
    inv = revenue * assumptions['cogs_pct'] * (assumptions['inv_days'] / 365)
    ap = revenue * assumptions['cogs_pct'] * (assumptions['ap_days'] / 365)
    nwc = ar + inv - ap
    
    delta_nwc = np.zeros(len(years))
    for i in range(1, len(years)):
        delta_nwc[i] = nwc[i] - nwc[i-1]
    
    # UFCF
    ufcf = nopat + da - capex - delta_nwc
    
    # Discount factors
    discount_factors = np.array([1 / (1 + wacc) ** i for i in range(len(years))])
    
    # PV of FCFs (years 1-5)
    pv_fcf = ufcf[1:] * discount_factors[1:]
    sum_pv_fcf = pv_fcf.sum()
    
    # Terminal Value
    terminal_fcf = ufcf[-1] * (1 + g)
    terminal_value = terminal_fcf / (wacc - g)
    pv_terminal = terminal_value * discount_factors[-1]
    
    # Enterprise Value
    enterprise_value = sum_pv_fcf + pv_terminal
    
    # Equity Value
    net_debt = assumptions['debt'] - 150  # Debt - Cash
    equity_value = enterprise_value - net_debt
    
    # Share Price
    share_price = equity_value / assumptions['shares_outstanding']
    
    return {
        'ufcf': ufcf,
        'pv_fcf': sum_pv_fcf,
        'terminal_value': terminal_value,
        'pv_terminal': pv_terminal,
        'enterprise_value': enterprise_value,
        'equity_value': equity_value,
        'share_price': share_price
    }

dcf = dcf_valuation(revenue, assumptions, years)

print("DCF VALUATION SUMMARY")
print("=" * 40)
print(f"PV of FCFs (Year 1-5):    ${dcf['pv_fcf']:,.0f}M")
print(f"Terminal Value:           ${dcf['terminal_value']:,.0f}M")
print(f"PV of Terminal Value:     ${dcf['pv_terminal']:,.0f}M")
print(f"Enterprise Value:         ${dcf['enterprise_value']:,.0f}M")
print(f"Equity Value:             ${dcf['equity_value']:,.0f}M")
print(f"Implied Share Price:      ${dcf['share_price']:,.2f}")

## 6. Sensitivity Analysis

In [ ]:
def sensitivity_analysis(revenue, assumptions, years):
    """WACC vs Terminal Growth sensitivity table"""
    
    wacc_range = [0.08, 0.09, 0.10, 0.11, 0.12]
    growth_range = [0.015, 0.020, 0.025, 0.030, 0.035]
    
    results = np.zeros((len(wacc_range), len(growth_range)))
    
    for i, wacc in enumerate(wacc_range):
        for j, g in enumerate(growth_range):
            temp_assumptions = assumptions.copy()
            temp_assumptions['wacc'] = wacc
            temp_assumptions['terminal_growth'] = g
            dcf = dcf_valuation(revenue, temp_assumptions, years)
            results[i, j] = dcf['enterprise_value']
    
    sens_df = pd.DataFrame(
        results,
        index=[f"{w:.0%}" for w in wacc_range],
        columns=[f"{g:.1%}" for g in growth_range]
    )
    sens_df.index.name = 'WACC / Growth'
    
    return sens_df

sensitivity = sensitivity_analysis(revenue, assumptions, years)
print("SENSITIVITY ANALYSIS - Enterprise Value (USD millions)")
print("=" * 60)
sensitivity.round(0)

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Revenue & Net Income
ax1 = axes[0, 0]
x = np.arange(len(years))
width = 0.35
ax1.bar(x - width/2, revenue, width, label='Revenue', color='#3b82f6')
ax1.bar(x + width/2, net_income, width, label='Net Income', color='#10b981')
ax1.set_xticks(x)
ax1.set_xticklabels(years)
ax1.set_title('Revenue vs Net Income', fontsize=12, fontweight='bold')
ax1.legend()
ax1.set_ylabel('USD millions')

# Margins
ax2 = axes[0, 1]
gross_margin = (revenue * (1 - assumptions['cogs_pct'])) / revenue * 100
ebit_margin = income_statement.loc['EBIT'].values / revenue * 100
net_margin = net_income / revenue * 100
ax2.plot(years, gross_margin, marker='o', label='Gross Margin', color='#3b82f6')
ax2.plot(years, ebit_margin, marker='s', label='EBIT Margin', color='#f59e0b')
ax2.plot(years, net_margin, marker='^', label='Net Margin', color='#10b981')
ax2.set_title('Profitability Margins', fontsize=12, fontweight='bold')
ax2.set_ylabel('%')
ax2.legend()
ax2.grid(alpha=0.3)

# Cash Flow Components
ax3 = axes[1, 0]
cfo_plot = [166, 180, 196, 212, 230]
cfi_plot = [-60, -65, -70, -75, -80]
cff_plot = [-71, -72, -74, -76, -78]
x_cf = np.arange(5)
ax3.bar(x_cf - 0.25, cfo_plot, 0.25, label='CFO', color='#10b981')
ax3.bar(x_cf, cfi_plot, 0.25, label='CFI', color='#ef4444')
ax3.bar(x_cf + 0.25, cff_plot, 0.25, label='CFF', color='#f59e0b')
ax3.set_xticks(x_cf)
ax3.set_xticklabels(years[1:])
ax3.set_title('Cash Flow Components', fontsize=12, fontweight='bold')
ax3.axhline(y=0, color='white', linewidth=0.5)
ax3.legend()
ax3.set_ylabel('USD millions')

# DCF Waterfall
ax4 = axes[1, 1]
categories = ['PV of FCFs', 'PV Terminal', 'EV', 'Net Debt', 'Equity']
values = [dcf['pv_fcf'], dcf['pv_terminal'], dcf['enterprise_value'], -50, dcf['equity_value']]
colors = ['#3b82f6', '#3b82f6', '#10b981', '#ef4444', '#10b981']
ax4.bar(categories, values, color=colors)
ax4.set_title('DCF Valuation Build-up', fontsize=12, fontweight='bold')
ax4.set_ylabel('USD millions')

plt.tight_layout()
plt.savefig('financial_model_charts.png', dpi=150, bbox_inches='tight', facecolor='#111827')
plt.show()

## Key Takeaways

1. **Fully Integrated Model**: Changes in assumptions automatically flow through all three statements
2. **Balance Sheet Balances**: Assets = Liabilities + Equity in all periods
3. **Cash Reconciliation**: Ending cash matches balance sheet position
4. **DCF Valuation**: $1.96B enterprise value, $19.11 implied share price
5. **Sensitivity**: Valuation most sensitive to terminal growth at lower WACC levels